In [8]:
import os 
import numpy as np
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split,cross_val_predict,KFold,StratifiedKFold,RepeatedKFold,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier

from  xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [9]:
class CFG:
    index = 'id'
    target = 'cost'
    random = 2023
    load_original = True,
    only_positive = True
    folds = 12
    
np.random.seed(CFG.random)

In [10]:
train_df = pd.read_csv('./train.csv')
train_df.head()

,id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
0,0,8.61,3.0,2.0,2.0,2.0,10.30,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0,62.09
1,1,5.00,2.0,4.0,0.0,3.0,6.66,1.0,0.0,1.0,28206.0,1.0,0.0,0.0,0.0,0.0,121.80
2,2,14.08,4.0,0.0,0.0,3.0,21.30,1.0,0.0,26.0,21215.0,1.0,0.0,0.0,0.0,0.0,83.51
3,3,4.02,3.0,5.0,0.0,0.0,14.80,0.0,1.0,36.0,21215.0,1.0,0.0,0.0,0.0,0.0,66.78
4,4,2.13,3.0,5.0,0.0,3.0,17.00,1.0,1.0,20.0,27694.0,1.0,1.0,1.0,1.0,1.0,111.51


In [11]:
train_df.dtypes

id                              int64
store_sales(in millions)      float64
unit_sales(in millions)       float64
total_children                float64
num_children_at_home          float64
avg_cars_at home(approx).1    float64
gross_weight                  float64
recyclable_package            float64
low_fat                       float64
units_per_case                float64
store_sqft                    float64
coffee_bar                    float64
video_store                   float64
salad_bar                     float64
prepared_food                 float64
florist                       float64
cost                          float64
dtype: object

In [19]:
test_df = pd.read_csv('./test.csv')

In [23]:
train_copy = train_df.copy()
y_tag = train_copy.pop(CFG.target).to_numpy()
numerical_cols = train_copy.columns

numerical_pipe = Pipeline([
    ('scaler',MinMaxScaler())
])

transforms = ColumnTransformer([
    ("num",numerical_pipe,numerical_cols)
])

train_copy = transforms.fit_transform(train_copy)
x_test = transforms.fit_transform(test_df)
print('TRAIN SHAPE :',train_copy.shape)
print('TEST SHAPE :', x_test.shape)

KeyError: 'cost'

In [16]:
models = []
rkf = RepeatedKFold(n_splits=CFG.folds,
                   random_state=CFG.random)

In [18]:
scores = []
for train_idx,val_idx in rkf.split(train_copy,y_tag):
    X_train,X_val = train_copy[train_idx],train_copy[val_idx]
    y_train,y_val = y_tag[train_idx],y_tag[val_idx]
    
    cat_m = CatBoostClassifier(iterations=150,verbose=0,random_state=CFG.random)
    cat_m.fit(X_train,y_train,verbose=False)
    models.append(cat_m)
    scores.append(roc_auc_score(y_val,cat_m.predict_proba(x_val)[:,1]))
print(f'Mean score: {np.mean(scores):.4f}')

CatBoostError: bad allocation